# Web Scraping

#### Introdução
- Com o mercado de trabalho em constante evolução, atrelado ao desenvolvimento acelerado das tecnologias atuais, é necessário que os profissionais sempre se mantenha antenados as exigências para se manterem atualizados. Olhando justamente para esse foco, o objetivo principal desse modelo é realizar a coleta dos dados de ofertas de emprego.

#### Índice
- [Bibliotecas importadas](#0)
- [Função de Importação](#1)
- [Coleta de Dados](#2)

<a id='0'><a/>
    
## Bibliotecas Importadas

In [1]:
import pandas as pd
import numpy as np
import re
import time
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


<a id='1'><a/>
## Função Operacional

In [2]:
class ImportarDados:
    def __init__(self):
        pass
    
    def pesquisar_vaga(self, job):
        """
        Método criado para acessar o link abaixo e procurar o cargo para a vaga solicitada
        
        """
        url = "https://www.glassdoor.com.br/Vaga/vagas.htm"
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--headless') # Exercutar o navegador sem interface gráfica.
        chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        driver = webdriver.Chrome(options=chrome_options) # Abrir o URL no navegador
        driver.get(url)                            
        time.sleep(4)
        # print(driver.page_source)
        
        elemento_pesquisa = driver.find_element(By.XPATH, '//*[@id="searchBar-jobTitle"]')
        elemento_pesquisa.send_keys(job)
        elemento_pesquisa.send_keys(Keys.RETURN)
        
        time.sleep(2)
        url_atual = driver.current_url
        driver.quit()
        return url_atual
    
    def coletar_html(self, url=None):
        """
        Método criado acessar o url recebido e extrair as informações das vagas presente no link.
        
        """
        if (url is None):
            job = input("Digite a vaga desejada ")
            url = ImportarDados().pesquisar_vaga(job)
            #url = f"{url}?sortBy=date_desc" # Por em ordem decrescente.
            
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--headless') # Exercutar o navegador sem interface gráfica.
        chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        
        driver = webdriver.Chrome(options=chrome_options) # Abrir o URL no navegador
        driver.get(url)
        time.sleep(2)
        botao_xpath = '//*[@id="left-column"]/div[2]/div/button'
        botao = driver.find_element(By.XPATH, botao_xpath)
        botao.click()
        
        codigo_html = driver.page_source
        driver.quit()
        soup = BeautifulSoup(codigo_html, 'html.parser')
        time.sleep(2)
    
        jobs = soup.find('ul', class_='JobsList_jobsList__Ey2Vo').find_all('li')
        return jobs
       
        
    def montar_dataframe(self):
        """
        Método criado para montar o dataframe das informações coletadas
        
        """
        jobs = ImportarDados().coletar_html()
        links = [jobs[i].find('a', class_='JobCard_seoLink__WdqHZ').get('href') for i in range(len(jobs) - 1)]
        tempo = [jobs[i].find('div', {'class': 'JobCard_listingAge__KuaxZ'}).text for i in range(len(jobs) - 1)]
        empresa = [jobs[i].find('div',class_='EmployerProfile_profileContainer__d5rMb').text for i in range(len(jobs) -1 )]
        cargo = [jobs[i].find('a',class_='JobCard_seoLink__WdqHZ').text for i in range(len(jobs) -1 )]
        local = [jobs[i].find('div',class_='JobCard_location__N_iYE').text for i in range(len(jobs) -1 )]
        description = list()
        for site in links:
            chrome_options = webdriver.ChromeOptions()
            chrome_options.add_argument('--headless') # Exercutar o navegador sem interface gráfica.
            chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
            
            driver = webdriver.Chrome(options=chrome_options) # Abrir o URL no navegador
            driver.get(site)
            time.sleep(1)
            codigo_html = driver.page_source
            soup = BeautifulSoup(codigo_html, 'html.parser')
            try:
                description.append(soup.find('div',class_='JobDetails_jobDescription__6VeBn JobDetails_blurDescription__fRQYh').text)
            except:
                description.append("não encontrado.")
                
        dados = [empresa, cargo, local, tempo, links, description]
        dados = pd.DataFrame(dados).T
        dados.columns = ['Empresa','Cargo','Local','Dias da postagem','Link','Descrição']
        dados['nota'] = dados['Empresa'].str.replace(',', '.').str.extract(r'(\d+\.\d+)')
        dados['nota'] = dados['nota'].fillna(10)
        dados['Empresa'] = [m[:len(m)-3] if n == '0' else m for n,m in zip(['0'  if i[-1].isdigit() else '1' for i in dados['Empresa']], dados['Empresa'])]
        return dados

<a id='2'></a>
## Coletando os dados

In [3]:
%%time
cientista = ImportarDados().montar_dataframe()

Digite a vaga desejada cientista de dados
CPU times: total: 1.17 s
Wall time: 8min 2s


In [4]:
cientista.head()

,Empresa,Cargo,Local,Dias da postagem,Link,Descrição,nota
0,C&A Brasil,Cientista de Dados Jr,Trabalho remoto,13d,https://www.glassdoor.com.br/job-listing/cient...,Trabalhamos para você ser mais você em todos o...,3.3
1,alt.bank,Pessoa Cientista de Dados,Trabalho remoto,4d,https://www.glassdoor.com.br/job-listing/pesso...,alt.bank é um player revolucionário no cenário...,4.0
2,innolevels,Cientista de dados,Trabalho remoto,7d,https://www.glassdoor.com.br/job-listing/cient...,não encontrado.,4.4
3,Brivia,Cientista de Dados,Trabalho remoto,mais de 30 dias,https://www.glassdoor.com.br/job-listing/cient...,Vaga para nosso BANCO DE TALENTOS\nProcuramos ...,4.0
4,Globo,Cientista de Dados | Big Data & AI,Trabalho remoto,mais de 30 dias,https://www.glassdoor.com.br/job-listing/cient...,A Globo é feita de gente que quer fazer difere...,4.2


<a id=4></a>
## Tratamento dos dados

In [76]:
cientista = pd.read_csv('./dados_empresa.csv')

In [77]:
cientista['Dias da postagem'] = (cientista['Dias da postagem'].apply(lambda x: '1d' if x.endswith('h') else x)
 .apply(lambda x: '30d' if x == 'mais de 30 dias' else x)
 .str.replace('d','').astype(int))

In [78]:
cientista['nota'] = cientista['nota'].astype(float)

In [79]:
def cargos(x):
    jr = ['júnior','jr','junior',' I']
    pl = ['pleno','PL',' II']
    sr = ['sr','sênior','senoir','III']
    if any(item in x.lower() for item in jr):
        return 'jr'
    elif any(item in x.lower() for item in pl):
        return 'pl'
    elif any(item in x.lower() for item in sr):
        return 'sr'
    else:
        return 'Não Especificado no título'
        
        
cientista['Cargo'] = cientista['Cargo'].apply(cargos)

In [80]:
cientista['Cargo'] = cientista['Cargo'].apply(lambda x: re.search(r'(Cientista de Dados|Analista de Dados|Engenheiro de Dados)|(\w+ \w+ Dados)', x).group(0) if re.search(r'(cientista de dados|analista de dados|engenheiro de dados)|(\w+ \w+ Dados)', x) else x)

In [81]:
cientista.head()

,Empresa,Cargo,Local,Dias da postagem,Link,Descrição,nota
0,C&A Brasil,jr,Trabalho remoto,13,https://www.glassdoor.com.br/job-listing/cient...,Trabalhamos para você ser mais você em todos o...,3.3
1,alt.bank,Não Especificado no título,Trabalho remoto,4,https://www.glassdoor.com.br/job-listing/pesso...,alt.bank é um player revolucionário no cenário...,4.0
2,innolevels,Não Especificado no título,Trabalho remoto,7,https://www.glassdoor.com.br/job-listing/cient...,não encontrado.,4.4
3,Brivia,Não Especificado no título,Trabalho remoto,30,https://www.glassdoor.com.br/job-listing/cient...,Vaga para nosso BANCO DE TALENTOS\nProcuramos ...,4.0
4,Globo,Não Especificado no título,Trabalho remoto,30,https://www.glassdoor.com.br/job-listing/cient...,A Globo é feita de gente que quer fazer difere...,4.2


In [82]:
cientista.to_csv("./dados_empresa_tratados.csv",index=False)